# 1. N-gram

In [ ]:
import nltk

def ngrams(sentence, n):
    words = sentence.split()
    ngrams = zip(*[words[i:] for i in range(n)])
    return list(ngrams)

sentence = "안녕하세요 만나서 진심으로 반가워요"

# 직접 구현한 ngram
unigram = ngrams(sentence, 1)
bigram = ngrams(sentence, 2)
trigram = ngrams(sentence, 3)

print(unigram)
print(bigram)
print(trigram)

# NLTK에서 지원하는 ngram
unigram = nltk.ngrams(sentence.split(), 1)
bigram = nltk.ngrams(sentence.split(), 2)
trigram = nltk.ngrams(sentence.split(), 3)

print(list(unigram))
print(list(bigram))
print(list(trigram))

# 동일하다

[('안녕하세요',), ('만나서',), ('진심으로',), ('반가워요',)]
[('안녕하세요', '만나서'), ('만나서', '진심으로'), ('진심으로', '반가워요')]
[('안녕하세요', '만나서', '진심으로'), ('만나서', '진심으로', '반가워요')]
[('안녕하세요',), ('만나서',), ('진심으로',), ('반가워요',)]
[('안녕하세요', '만나서'), ('만나서', '진심으로'), ('진심으로', '반가워요')]
[('안녕하세요', '만나서', '진심으로'), ('만나서', '진심으로', '반가워요')]


# 2. TF-IDF
Term Frequency-Inverse Document Frequency

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    "That movie is famous movie",
    "I like that actor",
    "I don’t like that actor"
]

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(corpus)
tfidf_matrix = tfidf_vectorizer.transform(corpus)

# 또는 fit_transorm 메서드로 학습과 변환 동시 수행
# tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

print(tfidf_matrix.toarray())
print(tfidf_vectorizer.vocabulary_)

[[0.         0.         0.39687454 0.39687454 0.         0.79374908
  0.2344005 ]
 [0.61980538 0.         0.         0.         0.61980538 0.
  0.48133417]
 [0.4804584  0.63174505 0.         0.         0.4804584  0.
  0.37311881]]
{'that': 6, 'movie': 5, 'is': 3, 'famous': 2, 'like': 4, 'actor': 0, 'don': 1}


# 3. Word2Vec

## 3.1. Skip-gram 모델


계층적 소프트맥스나 네거티브 샘플링과 같은 효율적인 기법을 적용하지 않은 기본 형식의 skip-gram 모델.  
단순히 입력 단어와 주변 단어를 룩업 테이블에서 가져와서 내적을 계산하고, 손실 함수를 최소화하는 방식으로 학습된다.

In [1]:
# 기본 skip-gram 클래스
from torch import nn

class VanillaSkipgram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings = vocab_size,
            embedding_dim = embedding_dim
        )
        self.linear = nn.Linear(
            in_features = embedding_dim,
            out_features = vocab_size
        )

    def forward(self, input_ids):
        embeddings = self.embedding(input_ids)
        output = self.linear(embeddings)
        return output

In [3]:
#!pip install Korpora
#!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached jpype1-1.6.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.0 kB)
Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
Using cached jpype1-1.6.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (495 kB)


테스트 세트를 불러오고, Okt 토크나이저로 형태소를 추출한다.

In [4]:
# 영화 리뷰 데이터세트 전처리
import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt

corpus = Korpora.load("nsmc")
corpus = pd.DataFrame(corpus.test)

tokenizer = Okt()
tokens = [tokenizer.morphs(review) for review in corpus.text]
print(tokens[:3])


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 118MB/s]                             
[nsmc] download ratings_test.txt: 4.90MB [00:00, 54.3MB/s]


[['굳', 'ㅋ'], ['GDNTOPCLASSINTHECLUB'], ['뭐', '야', '이', '평점', '들', '은', '....', '나쁘진', '않지만', '10', '점', '짜', '리', '는', '더', '더욱', '아니잖아']]


토큰화된 데이터를 활용해 단어 사전을 구축해보자.

In [5]:
# 단어 사전 구축
from collections import Counter

def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

vocab = build_vocab(corpus=tokens, n_vocab=5000, special_tokens=["<unk>"])
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<unk>', '.', '이', '영화', '의', '..', '가', '에', '...', '을']
5001


In [ ]:
# skip-gram의 단어 쌍 추출
def get_word_pairs(tokens, window_size):
    pairs = []
    for sentence in tokens:
        sentence_length = len(sentence)
        for idx, center_word in enumerate(sentence):
            window_start = max(0, idx - window_size)
            window_end = min(sentence_length, idx + window_size + 1)
            center_word = sentence[idx]
            context_words = sentence[window_start:idx] + sentence[idx+1:window_end]
            for context_word in context_words:
                pairs.append([center_word, context_word])
    return pairs

word_pairs = get_word_pairs(tokens, window_size=2)
print(word_pairs[:5])

[['굳', 'ㅋ'], ['ㅋ', '굳'], ['뭐', '야'], ['뭐', '이'], ['야', '뭐']]


출력 결과 각 단어 쌍은 [중심 단어, 주변 단어]로 구성돼 있다.  
임베딩 층은 단어의 인덱스를 입력으로 받기 때문에 단어 쌍을 인덱스 쌍으로 변환해야 한다.

In [ ]:
# 인덱스 쌍 변환
def get_index_pairs(word_pairs, token_to_id):
    pairs =  []
    unk_index = token_to_id["<unk>"]
    for word_pair in word_pairs:
        center_word, context_word = word_pair
        center_index = token_to_id.get(center_word, unk_index)
        context_index = token_to_id.get(context_word, unk_index)
        pairs.append([center_index, context_index])
    return pairs

index_pairs = get_index_pairs(word_pairs, token_to_id)
print(index_pairs[:5])
print(len(vocab))

[[595, 100], [100, 595], [77, 176], [77, 2], [176, 77]]
5001


이렇게 생성된 인덱스 쌍은 Skip-gram 모델의 임력 데이터로 사용된다.

In [ ]:
# 데이터로더 적용
import torch
from torch.utils.data import TensorDataset, DataLoader

index_pairs = torch.tensor(index_pairs)
center_indexes = index_pairs[:, 0]
context_indexes = index_pairs[:, 1]

dataset = TensorDataset(center_indexes, context_indexes)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

손실함수는 단어 사전 크기만큼 클래스가 있는 분류 문제이므로 크로스 엔트로피를 사용한다.  
크로스 엔트로피는 내부적으로 softmax 연산을 수행하기 때문에 신경망의 출력값을 후처리 없이 활용할 수 있다.

In [ ]:
# skip-gram 모델 준비 작업
from torch import optim

device = "cuda" if torch.cuda.is_available() else "cpu"
word2vec = VanillaSkipgram(vocab_size=len(token_to_id), embedding_dim=128).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(word2vec.parameters(), lr=0.1)

In [ ]:
# 모델 학습
for epoch in range(10):
    cost = 0.0
    for input_ids, target_ids in dataloader:
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)

        logits = word2vec(input_ids)
        loss = criterion(logits, target_ids)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(dataloader)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

Epoch :    1, Cost : 6.198
Epoch :    2, Cost : 5.983
Epoch :    3, Cost : 5.933
Epoch :    4, Cost : 5.903
Epoch :    5, Cost : 5.881
Epoch :    6, Cost : 5.863
Epoch :    7, Cost : 5.848
Epoch :    8, Cost : 5.835
Epoch :    9, Cost : 5.823
Epoch :   10, Cost : 5.813


모델이 간단한 구조이고, 데이터 수도 적은데 학습 시간이 굉장히 오래 걸린다.  
이럴 경우 계층적 소프트맥스나 네거티브 샘플링을 적용하면 시간을 단축할 수 있다.

In [ ]:
# 임베딩 값 추출
token_to_embedding = dict()
embedding_matrix = word2vec.embedding.weight.detach().cpu().numpy()

for word, embedding in zip(vocab, embedding_matrix):
    token_to_embedding[word] = embedding

index = 30
token = vocab[index]
token_embedding = token_to_embedding[token]

print(token)
print(token_embedding)

연기
[-0.84851855 -1.8222042   0.127349   -0.8652812  -1.0437642   0.3097375
 -2.31448     1.0563041  -0.6822946  -0.74907726  1.2799673   0.7929172
 -0.5319757  -1.7590579  -0.01364881  0.98411477 -1.0765609  -0.9145792
 -0.60660434 -0.24022445 -0.60561025  0.87058496 -1.2078757  -0.2727345
  0.26137322 -1.2436544  -0.03175946 -0.1551774  -0.9634912   0.5876038
 -0.6240664   0.3688197  -0.01440321 -0.39620957 -0.7398726  -0.45694444
  0.51004624 -0.07355792 -0.13838941  0.29571134 -0.6803196  -1.2643231
  0.18040971 -1.5091066   0.25738162 -0.24687687 -0.66137844  1.3742735
 -1.059737   -0.20769632  1.9868205  -0.02364214 -0.56665576 -1.8598207
 -0.5447131  -0.42990255  0.63883036 -0.27989024 -1.041601   -1.402175
  0.6961753  -0.43975404 -0.26018244 -0.50219434  1.0543333   0.40132272
 -1.3169492  -0.1853549   0.24995318 -0.6179749   0.8304516  -0.5887776
 -1.4016972   0.15095223  0.93971765 -0.13504054  0.22650304  0.894877
 -1.3905934  -0.50422287  0.9724714   0.65811294  1.4445307  

In [ ]:
# 단어 임베딩 유사도 계산 (코사인 유사도)
import numpy as np
from numpy.linalg import norm

def consine_similarity(a, b):
    cosine = np.dot(b, a) / (norm(b, axis=1) * norm(a))
    return cosine

def top_n_index(cosine_matrix, n):
    closest_indexes = cosine_matrix.argsort()[::-1]
    top_n = closest_indexes[1 : n + 1]
    return top_n

cosine_matrix = consine_similarity(token_embedding, embedding_matrix)
top_n = top_n_index(cosine_matrix, n=5)

print(f"{token}와 가장 유사한 5 개 단어")
for index in top_n:
    print(f"{id_to_token[index]} - 유사도 : {cosine_matrix[index]:.4f}")

연기와 가장 유사한 5 개 단어
탄 - 유사도 : 0.3392
잔 - 유사도 : 0.3288
앙 - 유사도 : 0.2936
좋았는데 - 유사도 : 0.2895
도통 - 유사도 : 0.2875


코사인 유사도는 두 벡터가 유사할수록 값이 1에 가까워지고, 다를수록 0에 가까워진다.

## 3.2. Gensim 라이브러리 기반 Word2Vec 모델
Pytorch보다 모델 학습이 훨씬 더 빠르다.

In [ ]:
#!pip install gensim

In [ ]:
import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt

corpus = Korpora.load("nsmc")
corpus = pd.DataFrame(corpus.test)

tokenizer = Okt()
tokens = [tokenizer.morphs(review) for review in corpus.text]

In [ ]:
# word2vec 모델 학습
from gensim.models import Word2Vec

word2vec = Word2Vec(
    sentences=tokens,
    vector_size=128,
    window=5,
    min_count=1,
    sg=1, # skip-gram모델 사용
    epochs=3,
    max_final_vocab=10000
)

In [ ]:
# 임베딩 추출 및 유사도 계산
word = "연기"
print(word2vec.wv[word])
print(word2vec.wv.most_similar(word, topn=5))
print(word2vec.wv.similarity(w1=word, w2="연기력"))

[ 0.05487183 -0.40850228  0.36768085  0.46263567 -0.087214    0.01316468
  0.1699383  -0.0036428  -0.5224892   0.32200712 -0.11613942 -0.19556259
 -0.02845111 -0.13054879 -0.29952967 -0.03536183 -0.22289719  0.55767673
 -0.17454194  0.5662799   0.6485446   0.63832265 -0.28808925 -0.12988247
 -0.10172804 -0.02290674 -0.48083398  0.13490091  0.20717323 -0.06965394
 -0.34435913  0.28688383  0.36697498 -0.02062768 -0.17954223 -0.3949535
  0.25773275 -0.2881663  -0.02794877 -0.2663027  -0.17277122  0.33320513
 -0.16124104 -0.41227168 -0.03408     0.18588974 -0.14622587 -0.16861063
  0.13592143  0.0870279   0.8002509   0.32507375  0.02429863  0.29627633
 -0.36664808 -0.31085148  0.2279023   0.2102413  -0.28402087  0.22580619
 -0.12934664 -0.06199892  0.09000939  0.19747588 -0.21389918 -0.0575014
 -0.09981064  0.32835373  0.08838161 -0.27330688 -0.30871361 -0.3007062
 -0.37315658  0.15201263 -0.23684643  0.17152165 -0.2528402  -0.40183845
 -0.15333372  0.40769902  0.04762038 -0.12024882  0.21

# 4. fastText 모델


In [ ]:
# KorNLI 데이터세트 전처리
from Korpora import Korpora

corpus = Korpora.load("kornli")
corpus_texts = corpus.get_all_texts() + corpus.get_all_pairs()
tokens = [sentence.split() for sentence in corpus_texts]

print(tokens[:3])


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : KakaoBrain
    Repository : https://github.com/kakaobrain/KorNLUDatasets
    References :
        - Ham, J., Choe, Y. J., Park, K., Choi, I., & Soh, H. (2020). KorNLI and KorSTS: New Benchmark
           Datasets for Korean Natural Language Understanding. arXiv preprint arXiv:2004.03289.
           (https://arxiv.org/abs/2004.03289)

    This is the dataset repository for our paper
    "KorNLI and KorSTS: New Benchmark Datasets for Korean Natural Language Understanding."
    (https://arxiv.org/abs/2004.03289)
    We introduce KorNLI and KorSTS, which are NLI and STS datasets in Korean.

    # License
    Creative Commons Attribution-ShareAlike license (CC BY-SA 4.0)
    Details in https://creativecommons.org/licenses

[kornli] download multinli.train.ko.tsv: 83.6MB [00:05, 16.2MB/s]                            
[kornli] download snli_1.0_train.ko.tsv: 78.5MB [00:00, 252MB/s]                            
[kornli] download xnli.dev.ko.tsv: 516kB [00:00, 32.5MB/s]
[kornli] download xnli.test.ko.tsv: 1.04MB [00:00, 1.65MB/s]                            


[['개념적으로', '크림', '스키밍은', '제품과', '지리라는', '두', '가지', '기본', '차원을', '가지고', '있다.'], ['시즌', '중에', '알고', '있는', '거', '알아?', '네', '레벨에서', '다음', '레벨로', '잃어버리는', '거야', '브레이브스가', '모팀을', '떠올리기로', '결정하면', '브레이브스가', '트리플', 'A에서', '한', '남자를', '떠올리기로', '결정하면', '더블', 'A가', '그를', '대신하러', '올라가고', 'A', '한', '명이', '그를', '대신하러', '올라간다.'], ['우리', '번호', '중', '하나가', '당신의', '지시를', '세밀하게', '수행할', '것이다.']]


In [ ]:
# fastText 모델 학습
from gensim.models import FastText

fastText = FastText(
    sentences=tokens,
    vector_size=128,
    window=5,
    min_count=5,
    sg=1, # skip-gram
    max_final_vocab=20000,
    epochs=3,
    min_n=2,
    max_n=6
)

In [ ]:
# OOV 처리
oov_token = "사랑해요"
oov_vector = fastText.wv[oov_token]

print(oov_token in fastText.wv.index_to_key)
print(fastText.wv.most_similar(oov_vector, topn=5))

False
[('사랑', 0.8876461982727051), ('사랑에', 0.8189547061920166), ('사랑의', 0.7929642200469971), ('사랑을', 0.7561405897140503), ('사랑하는', 0.7555779814720154)]
